In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import matplotlib.pylab as pylab

pylab.rcParams['figure.figsize'] = 15, 10

## Strategy

In [ ]:
class myStrategy(bt.Strategy):
    '''
    Implementation of my strategy. Strategy parameters below, optimize them to get the 
    best possible implementation of my strategy given historical data. 
    '''
    params = (
        ('maperiod', 25),
        ('momperiod',15),
        ('printlog',True),
    )

    def log(self, txt, dt=None, doprint=False):
        '''
        Simple logging function. 
        '''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        #self.hold_limit = self.params.hold_limit
        
        self.mom = bt.indicators.Momentum(self.datas[0],period = self.params.momperiod)
        self.sma = bt.indicators.SMA(self.mom, period = self.params.maperiod)

        self.long = bt.indicators.CrossUp(self.mom, self.sma)
        self.short = bt.indicators.CrossDown(self.mom, self.sma)
        
        self.closeshort = self.long 
        self.closelong = self.short 


    def notify_order(self, order):
        '''
        Keeps track of buy/sell executions.
        '''
        if order.status in [order.Submitted, order.Accepted]:
            return
        

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        '''
        Keeps track of completed trades, ie: one round-trip.
        '''
        
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    
    def next(self):
        if self.order:
            return

        if not self.position:
            if self.long and (self.dataclose[0] > self.dataopen[0]):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
            elif self.short and (self.dataclose[0] < self.dataopen[0]):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
                
        else:
            #pos_length = len(self) - self.bar_executed + 1
            
            if (self.position.size > 0) and self.closelong:
                self.order = self.close()
                
            elif (self.position.size < 0) and self.closeshort:
                self.order = self.close()    
                
    

    def stop(self):
        self.log('(MA Period {}, MOM Period {}) - Ending Value {}'.format(self.params.maperiod, self.params.momperiod,  self.broker.getvalue()), doprint=True)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.maperiod = self.strategy.params.maperiod
        self.momperiod = self.strategy.params.momperiod

        self.equity = []

    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0
        self.first_trade_open = None
        
    def notify_trade(self, trade):
        if trade.isclosed:
            self.ntrade += 1
            self.equity.append(self.strategy.broker.getvalue())
            
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        equity = np.asarray([self.init_cash,] + self.equity)

        outp = {
            'params': (self.maperiod,self.momperiod),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade,
            'trades': np.diff(equity).tolist()
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    #print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

## In-sample Parameters
Basically in-sample optimization for each time period

In [ ]:
# one year
periods = [
{'run': 0, 'oos': ('2017-04-01', '2017-05-01'), 'is': ('2017-01-01', '2017-04-01')},
{'run': 1, 'oos': ('2017-05-01', '2017-06-01'), 'is': ('2017-02-01', '2017-05-01')},
{'run': 2, 'oos': ('2017-06-01', '2017-07-01'), 'is': ('2017-03-01', '2017-06-01')},
{'run': 3, 'oos': ('2017-07-01', '2017-08-01'), 'is': ('2017-04-01', '2017-07-01')},
{'run': 4, 'oos': ('2017-08-01', '2017-09-01'), 'is': ('2017-05-01', '2017-08-01')},
{'run': 5, 'oos': ('2017-09-01', '2017-10-01'), 'is': ('2017-06-01', '2017-09-01')},
{'run': 6, 'oos': ('2017-10-01', '2017-11-01'), 'is': ('2017-07-01', '2017-10-01')},
{'run': 7, 'oos': ('2017-11-01', '2017-12-01'), 'is': ('2017-08-01', '2017-11-01')},
{'run': 8, 'oos': ('2017-12-01', '2018-01-01'), 'is': ('2017-09-01', '2017-12-01')},
{'run': 9, 'oos': ('2018-01-01', '2018-02-01'), 'is': ('2017-10-01', '2018-01-01')},
{'run': 10, 'oos': ('2018-02-01', '2018-03-01'), 'is': ('2017-11-01', '2018-02-01')},
{'run': 11, 'oos': ('2018-03-01', '2018-04-01'), 'is': ('2017-12-01', '2018-03-01')},
{'run': 12, 'oos': ('2018-04-01', '2018-05-01'), 'is': ('2018-01-01', '2018-04-01')},
]

In [ ]:
best_is_params = []
for oos_is in periods:
    start_date, end_date = oos_is['is']
    run_num = oos_is['run']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        myStrategy,
        maperiod=(20,25,30,35),
        momperiod=(5,7,9,11),
        printlog=False)
    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, best_profit, best_ntrades, best_equity = best_result_from_cerebro_opti_run(result)

    best_is_params.append({'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_ntrades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     })
    

print('The best IS parameters are: {}'.format(pformat(best_is_params)))

In [ ]:
trades = []
for i in range(len(best_is_params)):
    trades.append(best_is_params[i]['train_tradeslist'])
    
equity_sum = 0
equity = []
trades = [trade for period in trades for trade in period]
for i in range(len(trades)):
    equity_sum += trades[i]
    equity.append(equity_sum)
    
plt.plot(equity)
plt.title("WF Equity Curve")

## Testing best IS parameters OOS

In [ ]:
run_num = 0
for best in best_is_params:
    start_date, end_date = best['test_period']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    cerebro.addstrategy(myStrategy,
        maperiod=best['train_param'][0],
        momperiod=best['train_param'][1])

    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    r = result[0].analyzers.decisive.get_analysis()
    best_is_params[run_num]['test_numtrades'] = r['ntrade']
    best_is_params[run_num]['test_tradeslist'] = r['trades']
    best_is_params[run_num]['test_profit'] = r['profit']
    run_num += 1

## Walkforward Equity Curve
Walkforward is basically time dependent optimization. We use the best in sample parameters of each period. 

In [ ]:
class WalkforwardStrategy(bt.Strategy):

    params = (
        ('maperiod', 25),
        ('momperiod',15),
        ('printlog',False),
        ('live', False),
        ('walkforward', None)
    )
        
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        #self.hold_limit = self.params.hold_limit
        
        self.mom = bt.indicators.Momentum(self.datas[0],period = self.params.momperiod)
        self.sma = bt.indicators.SMA(self.mom, period = self.params.maperiod)

        self.long = bt.indicators.CrossUp(self.mom, self.sma)
        self.short = bt.indicators.CrossDown(self.mom, self.sma)
        
        self.closeshort = self.long 
        self.closelong = self.short 
        
        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        self.current_row = None
        self.wfparams = []
        
        #self.params.walkforward = periods
        if self.params.walkforward:
            #print(self.params.walkforward)
            for run in self.params.walkforward:
                self.wfparams.append({
                    'sma': bt.indicators.SMA(self.datas[0], period= run['train_param'][0]),
                    'mom': bt.indicators.Momentum(self.datas[0], period = run['train_param'][1]),
                    'train_param': run['train_param'],
                    'test_period': run['test_period']
                }) 

        print(pformat(self.wfparams))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        for row in self.wfparams:
            start, end = row['test_period']
            period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            
            if self.long and (self.dataclose[0] > self.dataopen[0]):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
            elif self.short and (self.dataclose[0] < self.dataopen[0]):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
                
        else:
            if (self.position.size > 0) and self.closelong:
                self.order = self.close()
                
            elif (self.position.size < 0) and self.closeshort:
                self.order = self.close()    
                
    def stop(self):
        self.log('(MA Period %2d, MOM %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.params.momperiod, self.broker.getvalue()), doprint=True)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    maperiod=25,
                    momperiod = 15,
                    walkforward=best_is_params,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = periods[0]['oos'][0]
end_date = periods[-1]['oos'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

In [ ]:
r = results[0].analyzers.decisive.get_analysis()
cumsum = np.asarray(r['trades']).cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

cerebro.plot(volume=False, iplot=False)

Average your ratios of annualized OOS/IS performance for all runs (Walk Forward Efficiency)

In [ ]:
def get_num_days(interval):
    start = datetime.datetime.strptime(interval[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(interval[1], '%Y-%m-%d')
    num_days = (end - start).days
    return num_days

In [ ]:
oos_trades = []
wfe_list = []
for values in best_is_params:
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    is_days = get_num_days(values['train_period'])
    oos_days = get_num_days(values['test_period'])

    is_annual_profit = 365/is_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / is_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(run_num, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(is_days, oos_days,
                                                                            is_annual_profit, oos_annual_profit))

avg_wfe = np.round(np.asarray(wfe_list).mean(),1)

print("\n\n")
print("########################################################")

print('Average WFE: {}%'.format(avg_wfe))